#Upload Data Set




In [1]:
!pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d datatattle/covid-19-nlp-text-classification

  0% 0.00/4.38M [00:00<?, ?B/s]
100% 4.38M/4.38M [00:00<00:00, 71.7MB/s]


In [6]:
!unzip \*.zip && rm *.zip

Archive:  covid-19-nlp-text-classification.zip
  inflating: Corona_NLP_test.csv     
  inflating: Corona_NLP_train.csv    


###Adding Required Libraries

In [7]:
import pandas as pd

###Load Data Set

In [27]:
train = pd.read_csv("Corona_NLP_train.csv",encoding='ISO-8859-1')
test = pd.read_csv("Corona_NLP_test.csv",encoding='ISO-8859-1')

In [9]:
train.info() #check

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


#Normalization 2

1.  Deleting Duplicate
2.  Deleting Unnecessary Columns  
3.  Normalizing Tweet
*   #covid19 ---> < hashtag> covid19 < /hashtag>
*   COVID ---> < allcaps> covid < /allcaps>
*   @aydcsr ---> < user>
*   http.... ---> < url>
*   19 ---> < number> 
4.  Lowercase
5.  3 and 5 Sentiment Datasets

###Adding Required Libraries

In [10]:
!pip install ekphrasis

     |████████████████████████████████| 80 kB 3.8 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-py3-none-any.whl size=82842 sha256=c11e4d5ecf87d9bc4bddff8522e9404351904dc11a7a2951c3ddff4362f209f9
  Stored in directory: /root/.cache/pip/wheels/f7/ec/0d/12659e32faf780546945d0120f2c8410eb3efb7426731da88f
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=c1a958dd665e09edaa3b190632f1e2db243a3ccad332ad7abb2f12ed19487443
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built ekphrasis ftfy


In [11]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer

### Deleting Duplicate Tweets

In [14]:
train.drop_duplicates(subset="OriginalTweet",inplace=True)
test.drop_duplicates(subset="OriginalTweet",inplace=True)

### Deleting Unnecessary Columns

In [28]:
train = train[["OriginalTweet","Sentiment"]]
test = test[["OriginalTweet","Sentiment"]]

### Normalization

In [31]:
text_processor = TextPreProcessor(
# terms that will be normalized
normalize=['url', 'email', 'user', 'url', 'date', 'number','money'],
# terms that will be annotated
annotate={"hashtag", "allcaps"},
fix_html=True, # fix HTML tokens
# corpus from which the word statistics are going to be used
# for word segmentation
segmenter="twitter",
# corpus from which the word statistics are going to be used
# for spell correction
corrector="twitter",
unpack_hashtags=True, # perform word segmentation on hashtags
unpack_contractions=True, # Unpack contractions (can't -> can not)
spell_correct_elong=False, # spell correction for elongated words
# select a tokenizer. You can use SocialTokenizer, or pass your own
# the tokenizer, should take as input a string and return a list of tokens
tokenizer=SocialTokenizer(lowercase=True).tokenize,
)

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


####Train

In [21]:
train.head(300)

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...
295,Oregon Gov. Kate Brown on Monday afternoon ord...,Negative
296,#Coronavirus - #Europe calls for calm as food ...,Negative
297,When even Amazon is struggling to deliver food...,Extremely Negative
298,Anthony Fauci said that while some people may ...,Negative


In [29]:
for i in range(len(train)):
  train.iloc[i]["OriginalTweet"] = " ".join(text_processor.pre_process_doc(train.iloc[i]["OriginalTweet"] ))

In [30]:
train.head(300)

,OriginalTweet,Sentiment
0,<user> <user> <user> <url> and <url> and <url>,Neutral
1,advice talk to your neighbours family to excha...,Positive
2,coronavirus australia : woolworths to give eld...,Positive
3,my food stock is not the only one which is emp...,Positive
4,"me , ready to go at supermarket during the <ha...",Extremely Negative
...,...,...
295,oregon gov . kate brown on monday afternoon or...,Negative
296,<hashtag> coronavirus </hashtag> - <hashtag> e...,Negative
297,when even amazon is struggling to deliver food...,Extremely Negative
298,anthony fauci said that while some people may ...,Negative


####Test

In [32]:
test.head(300)

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral
...,...,...
295,The Coronavirus virus has people buying up all...,Extremely Positive
296,"retail notes growth in demand for groceries, c...",Positive
297,#Covid_19 stole Bernie's platform- it lowered ...,Positive
298,I have PTSD from surviving Hurricane Katrina.\...,Positive


In [33]:
for i in range(len(test)):
  test.iloc[i]["OriginalTweet"] = " ".join(text_processor.pre_process_doc(test.iloc[i]["OriginalTweet"] ))

In [34]:
test.head(300)

,OriginalTweet,Sentiment
0,<allcaps> trending </allcaps> : new yorkers en...,Extremely Negative
1,when i could not find hand sanitizer at fred m...,Positive
2,find out how you can protect yourself and love...,Extremely Positive
3,<hashtag> panic </hashtag> buying hits <hashta...,Negative
4,<hashtag> toilet paper </hashtag> <hashtag> du...,Neutral
...,...,...
295,the coronavirus virus has people buying up all...,Extremely Positive
296,"retail notes growth in demand for groceries , ...",Positive
297,<hashtag> covid 19 </hashtag> stole bernie ' s...,Positive
298,i have <allcaps> ptsd </allcaps> from survivin...,Positive


###Creating 3 Sentiment Dataset

In [35]:
train_3s = train.copy()
test_3s = test.copy()

In [37]:
five2three = {"Extremely Positive":"Positive","Positive":"Positive","Neutral":"Neutral","Negative":"Negative","Extremely Negative":"Negative"}
train_3s["Sentiment"] = [five2three[x] for x in train_3s["Sentiment"]]
test_3s["Sentiment"] = [five2three[x] for x in test_3s["Sentiment"]]

In [40]:
train_3s["Sentiment"].value_counts()

Positive    18046
Negative    15398
Neutral      7713
Name: Sentiment, dtype: int64

In [41]:
test_3s["Sentiment"].value_counts()

Negative    1633
Positive    1546
Neutral      619
Name: Sentiment, dtype: int64

#Save Data Sets

In [42]:
train.to_csv("COVID19_train_N2_S5.csv", index = False)
test.to_csv("COVID19_test_N2_S5.csv", index = False)
train_3s.to_csv("COVID19_train_N2_S3.csv",index = False)
test_3s.to_csv("COVID19_test_N2_S3.csv", index = False)